In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
import random
import datetime

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 8, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(8, 4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(4 * 75 * 100, 64),
    nn.Linear(64, 5))

In [4]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [5]:
data_path = '../input/cassava-2/cassava_2.pt'
loaded_model = model.to(device=device)
loaded_model.load_state_dict(torch.load(data_path, map_location=device))


<All keys matched successfully>

In [6]:
# df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
# df.head(2)

In [7]:
images_ids = []
labels_pred = []
import os
for dirname, _, filenames in os.walk('../input/cassava-leaf-disease-classification/test_images'):
    for filename in filenames:
        pic = os.path.join(dirname, filename)
        to_tensor = transforms.ToTensor()
        x = to_tensor(Image.open(pic))
        tens2 = x.unsqueeze(0)

        with torch.no_grad():
            loaded_model.eval()
            tens2 = tens2.to(device=device) 
            y_pred = model(tens2)
            ind, pred = torch.max(y_pred, dim=1)
            images_ids.append(filename)
            labels_pred.append(str(int(pred)))

In [8]:
df2 = pd.DataFrame({'image_id': images_ids})
df2['label'] = labels_pred
df2.to_csv('submission.csv', index=False)